# Case de Estudos: Santander Value Prediction

Ajude Santander a identificar o valor das transações para cada cliente potencial. Esse é um primeiro passo que o Santander precisa acertar para personalizar seus serviços em grande escala.
De acordo com uma pesquisa da Epsilon, 80% dos clientes tendem a voltar a fazer negócios com a sua empresa se a mesma entregar um serviço personalizado.

<br>
## Link para os dados e o desafio: 

https://www.kaggle.com/c/santander-value-prediction-challenge/data

O case podera ser quebrado nas 6 partes seguintes:

    Identificar o problema
        Qual o tipo de problema(classificação, regressão, clustering)?
    Necessidades de aplicar transformaçoes?
        Ex: imputing de valores null, encoding de colunas string, etc
    Separar os sets de treinamento e teste
    Baseline
        Achar uma baseline, um primeiro modelo para ter uma referencia
    Escolher a metrica
    Melhorar o resultado
        Feature engineering, otimizaçao do modelo, hiperparametros, etc



In [1]:
#Criação e manipulação dos dataframes
import pandas as pd
pd.set_option('display.max_rows', 500)

#Operações matemáticas
import numpy as np

#Separar o dataframe em treino e teste
from sklearn.model_selection import train_test_split, GridSearchCV

#Métricas de avaliação do modelo
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

#Regressores utilizados
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.neural_network import MLPRegressor 
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor

#Normalização dos dados 
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

#Ignorar os avisos do sklearn
import warnings
warnings.filterwarnings('ignore')

#Verificar o tempo de run de cada modelo 
from time import time

#df_pred = pd.DataFrame({'Valores Reais': y_test, 'Valores preditos': y_pred})
#df_pred

In [2]:
df = pd.read_csv('train.csv')

display(df)

,ID,target,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
0,000d6aaf2,38000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
1,000fbd867,600000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
2,0027d6b71,10000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
3,0028cbf45,2000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
4,002a68644,14400000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4454,ff85154c8,1065000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
4455,ffb6b3f4f,48000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,80000.0,0,0,0,0,0,0,0
4456,ffcf61eb6,2800000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
4457,ffea67e98,10000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0


In [3]:
#Verificando o uso de memória do dataframe
df.memory_usage().sum()/(1024*1024*1024)

0.16587825864553452

In [4]:
def verificar_nulos(df):
    i = 0
    for coluna in df.columns:
        if df[coluna].isnull().sum() > 0:
            print(f'A coluna {coluna} possui {df[coluna].isnull().sum()} valores nulos.')
        
        if df[coluna].isnull().sum() == 0:
            i += 1
    
    if i == df.shape[1]:
        print('O dataframe não possui nenhum valor nulo')

verificar_nulos(df)

O dataframe não possui nenhum valor nulo


## Separando o dataset em treino e teste

In [5]:
y = df['target']
X = df.drop(['ID', 'target'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

## Normalizando os dados com o Standard Scaler

In [ ]:
standardscaler = StandardScaler().fit(X_train, X_test)
X_train = standardscaler.transform(X_train)
X_test = standardscaler.transform(X_test)

## Normalizando os dados com o MinMaxScaler

In [ ]:
minmaxscaler = MinMaxScaler().fit(X_train, X_test)
X_train = minmaxscaler.transform(X_train)
X_test = minmaxscaler.transform(X_test)

## Normalizando os dados com o Robust Scaler

In [6]:
robustscaler = RobustScaler().fit(X_train, X_test)
X_train = robustscaler.transform(X_train)
X_test = robustscaler.transform(X_test)

## Testando os modelos

In [7]:
#Criando a lista dos modelos que serão testados
models = [
    LinearRegression(),
    Ridge(),
    Lasso(),
    RandomForestRegressor(),
    MLPRegressor(),
    DecisionTreeRegressor(),
    KNeighborsRegressor(),
    SVR(),
    GaussianProcessRegressor()
]

#Criando as listas vazias das métricas do modelo
MAE = []
RMSE = []
MSE = []
r2 = []
names = []
parameters = []
tempo = []

#Testando modelo por modelo
for model in models:
    #Startando o cronometro para ver o tempo de cada modelo 
    tempo_inicial = time()
    
    #Chamando o regressor e fazendo o predict
    regressor = model.fit(X_train, y_train)
    y_pred = regressor.predict(X_test)
    
    #Adicionando o valor da métrica de cada modelo para criar o dataframe depois
    MAE.append(mean_absolute_error(y_test, y_pred))
    RMSE.append(np.sqrt(mean_squared_error(y_test, y_pred)))
    MSE.append(mean_squared_error(y_test, y_pred))
    r2.append(r2_score(y_test, y_pred))
               
    #Para pegar o nome do modelo basta pegar o nome da clas
    names.append(model.__class__.__name__)
               
    #Para pegar os parametros utilizados
    parameters.append(model.get_params())
               
    #Parando o cronometro para ver o tempo e adicionando na lista 
    tempo_final = time() - tempo_inicial
    tempo.append(tempo_final)
               
               
#Criando o dataframe com as métricas de todos os modelos 
resultado = pd.DataFrame({'Nome': names,
                         'Parametros': parameters,
                         'MAE': MAE,
                         'RMSE': RMSE,
                         'MSE': MSE,
                         'R2': r2,
                         'Run Time': tempo})


In [8]:
display(resultado)

,Nome,Parametros,MAE,RMSE,MSE,R2,Run Time
0,LinearRegression,"{'copy_X': True, 'fit_intercept': True, 'n_job...",3.722582e+14,7.980047e+14,6.368116e+29,-1.122936e+16,14.665339
1,Ridge,"{'alpha': 1.0, 'copy_X': True, 'fit_intercept'...",2.281779e+10,4.987457e+10,2.487472e+21,-4.386341e+07,8.784947
2,Lasso,"{'alpha': 1.0, 'copy_X': True, 'fit_intercept'...",2.470443e+08,2.302449e+09,5.301271e+18,-9.348017e+04,14.643559
3,RandomForestRegressor,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri...",4.598845e+06,6.658703e+06,4.433833e+13,2.181500e-01,492.912372
4,MLPRegressor,"{'activation': 'relu', 'alpha': 0.0001, 'batch...",6.948808e+06,1.394988e+07,1.945993e+14,-2.431510e+00,51.423721
5,DecisionTreeRegressor,"{'ccp_alpha': 0.0, 'criterion': 'squared_error...",6.241866e+06,1.026487e+07,1.053676e+14,-8.580229e-01,9.646216
6,KNeighborsRegressor,"{'algorithm': 'auto', 'leaf_size': 30, 'metric...",4.925915e+06,7.665000e+06,5.875222e+13,-3.602068e-02,0.311838
7,SVR,"{'C': 1.0, 'cache_size': 200, 'coef0': 0.0, 'd...",4.785519e+06,8.238722e+06,6.787654e+13,-1.969164e-01,43.455495
8,GaussianProcessRegressor,"{'alpha': 1e-10, 'copy_X_train': True, 'kernel...",5.597790e+06,9.383217e+06,8.804476e+13,-5.525573e-01,65.099092


## Tuning dos hiperparametros 

In [10]:
#Utilizando o SelectKBest para achar o melhor valor para K
sel_kbest = SelectKBest(f_regression, k=45).fit(X_train, y_train)

X_train_sel = sel_kbest.transform(X_train)
X_test_sel = sel_kbest.transform(X_test)

print(f'Shape do X_train: {X_train_sel.shape}')

#Verificar o MAE uitilizando o SelectKbest
reg = LinearRegression().fit(X_train_sel, y_train)
y_train_pred = reg.predict(X_train_sel)
y_pred = reg.predict(X_test_sel)

print('Modelo selecionando os melhores valores para K')
print(mean_absolute_error(y_train, y_train_pred))
print(mean_absolute_error(y_test, y_pred))

Shape do X_train: (3567, 45)
Modelo selecionando os melhores valores para K
5322548.588995631
5230263.09906781


In [ ]:
#Melhorar o resultado 
sel_kbest = SelectKBest(f_regression, k=45).fit(X_train, y_train)
X_train_sel = sel_kbest.transform(X_train)
X_test_sel = sel_kbest.transform(X_test)

regr = RandomForestRegressor(max_depth=6, random_state=0)
regr.fit(X_train_sel, y_train)

y_train_pred = regr.predict(X_train_sel)
y_pred = regr.predict(X_test_sel)

print(mean_absolute_error(y_train, y_train_pred))
print(mean_absolute_error(y_test, y_pred))

In [ ]:
#Utilizando o GridSearchCV para encontrar os melhores valores para os parametros
parameters = {'max_depth':[2,6,10,20,30,40],
              'n_estimators':[20,50,100,200,300,500],
              'max_features':['sqrt', 'log2']}

regr = RandomForestRegressor(random_state=0, n_jobs=-1, verbose=1)

clf = GridSearchCV(regr, parameters)
clf.fit(X_train_sel, y_train)

y_train_pred = clf.predict(X_train_sel)
y_pred = clf.predict(X_test_sel)

print('MAE do modelo utilizando gridsearchCV')
print(mean_absolute_error(y_train, y_train_pred))
print(mean_absolute_error(y_test, y_pred))
print(clf.best_estimator_)